Note:
    
    1. When listing info about folders and files the level should be INFO.
    
    2. When the passing value for folder or file is not valid (folder or file doesn’t exist) the level should be WARNING.
    
    3. Any computation errors should be captured (e.g. total storage of files, folders, drives cannot be negative) and classified under the ERROR level.
    
    4. When Python can NOT read the OS structure, the level should be CRITICAL.
    
    5. Any other activity/message of the logger should have a level of DEBUG.

In [6]:
# PURP: - TO HANLDE INITAL LOGGING AND ARGS LOGIC. 
import logging
import argparse
from constants import D, DRV, L, FLD, F, FIL, T, TYP, LOG_FORMAT, LOG_DATE_FORMAT


# MARK:- Logger Initilized
logging.basicConfig(filename='dump.log', level=logging.DEBUG, format=LOG_FORMAT, datefmt=LOG_DATE_FORMAT)
logging.debug("Logging Initilized")

# MARK:- Set up Argparse paramaeters
parser = argparse.ArgumentParser()
parser.add_argument(D, help='list all the drives of the machine with additional info')
parser.add_argument(DRV, help='list additional info of single drive')

parser.add_argument(L, help='list all the folders of the machine with additional info:')
parser.add_argument(FLD, help='list additional info of single folder')

parser.add_argument(F, help='list all the files of the machine with additional info')
parser.add_argument(FIL, help='list additional info of single file')

parser.add_argument(T, help='list all the types of files of the machine with additional info')
parser.add_argument(TYP, help='list additional info of type of single file')
logging.debug("Args Added")

args = parser.parse_args(args=['-d', 'D'])

List Drives

    ‘-d’: To list all the drives of the machine with the following info:
        - Drive name / letter
        - Total number of directories
        - Total number of files
        - Total allocated, used, free storage
    A positional argument ‘drv’ that will indicate the name of a single drive to report the above info.

In [7]:
from drive_dump import dump_drives, dump_drive
from info_logger import log_drive_info

# MARK:- Begin Args Parameters Handeling
logging.debug("Beginning Args Handeling...")
# MARK:- -d args
if args.d:
    logging.debug('-d argument was passed')
    logging.debug("Dumping Local Computer Drives info")
    try:
        drives = dump_drives()
        for drive in drives:
            drive_name, num_of_files_and_dirs, storage = drive
            log_drive_info(drive_name, num_of_files_and_dirs, storage)
    except Exception:
        logging.error("Could Not Dump Drive Info")

# MARK:- -drv args
if args.drv:
    logging.debug('-drv argument was passed')
    logging.debug("Dumping info of %s Drive" % args.drv)
    drive = dump_drive(args.drv)
    if drive is None:
        logging.critical('Program was unable to process %s path' % args.drv)
    else:
        drive_name, num_of_files_and_dirs, storage = drive
        log_drive_info(drive_name, num_of_files_and_dirs, storage)

Lists Folders

    l’: To list all the folders in a given drive with the following info:
        - Folder name
        - Total number of files per folder
        - Total storage used per folder and sum of all storage for all folders.
    A positional argument ‘fld’ that will pass the folder name to report the above info.

In [49]:
from os import walk, path
import shutil
from drive_dump import get_disk_info
from helper import format_size
from constants import COMP_ERROR
import logging

def get_folder_and_num_of_files(directory):
    
    files = 0
    folders = []
    try:
        for pathname, dirnames, _ in walk(directory):
            for folder in dirnames:
                for _, _, filename in walk(path.join(pathname, folder)):
                    files += len(filename)
                    folders.append((folder,files))
                    files = 0
                    break 
            break
    except Exception:
        logging.critical("Cannot get count of drive's folder files and folders")
        return [] # empty lists I work out null handling
    if files < 0:
        logging.error("File count less than 0")
    if len(folders) < 0:
        logging.error("Folder count less than 0")

    return folders

folders = []
for folder in get_folder_and_num_of_files('/Volumes'):
    foldername, num_of_files = folder
    folder += (get_disk_info('/Volumes/' + foldername),)
    folders.append(folder)
print(folders)



[('Macintosh HD', 2, ('926.35 GB', '18.02 GB', '782.37 GB'))]


Lists Files

    ‘-f’: To list all the files of the machine with the following info:
        - File name
        - File type (‘py’, ‘ipynb’, ‘exe’, ‘txt’, ‘csv’, ‘pdf’, ‘other’)
        - File size
        - Date/Time stamp of the file
    A positional argument ‘fil’ that will pass the file name to report the above info.

Lists File Types

    ‘-t’: To list all the types of files exist in the machine with the following info:
        - File type
        - Total number of files per file type
        - Total storage used per file type
    A positional argument ‘typ’ that will indicate the type of file to report the above info.